# Performance comparaison

| Fingerprint | R2 | RMSE |  
|:-:|:-:|:-:|  
| ECFP| 0.765 | 0.9808 |
|Can2Can|0.7176|1.073|
| NFP| 0.8845 | 0.6868 |

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

import chainer
from seq2seq import Seq2seq, load_vocabulary

UNK, EOS = 0, 1

In [2]:
df_train = pd.read_csv('../data/sol_train.csv')
df_test = pd.read_csv('../data/sol_test.csv')
df_train.head()

,Unnamed: 0,SMILES,unknown,solubility,processed_smiles
0,418,[nH0]1c(SC)c2c([nH0]cc[nH0]2)[nH0]c1,6966-78-5,-2.36,[ n H 0 ] 1 c ( S C ) c 2 c ( [ n H 0 ] c c [ ...
1,342,CCC(C)Cl,78-86-4,-1.96,C C C ( C ) Cl
2,1058,O=C(NC(=O)c1ccccc1)c1ccccc1,614-28-8,-2.27,O = C ( N C ( = O ) c 1 c c c c c 1 ) c 1 c c ...
3,113,CC(C(C)(C)C)O,464-07-3,-0.62,C C ( C ( C ) ( C ) C ) O
4,294,[O-][N+](c1c(O)cccc1)=O,88-75-5,-1.74,[ O- ] [ N+ ] ( c 1 c ( O ) c c c c 1 ) = O


In [3]:
x_train = df_train['processed_smiles']
y_train = df_train['solubility']
x_test = df_test['processed_smiles']
y_test = df_test['solubility']

In [4]:
def load_data(vocabulary, lst):
    data = []
    for l in lst:
        words = l.strip().split()
        array = np.array([vocabulary.get(w, UNK) for w in words], np.int32)
        data.append(array)
    return data

In [5]:
source_ids = load_vocabulary('../data/vocab.txt')
xnum_train = load_data(source_ids, x_train)
xnum_test = load_data(source_ids, x_test)

# Encode to fingerprint

In [6]:
model = Seq2seq(1, 46, 46, 256)
chainer.serializers.load_npz('../result/can2can_iter_132000.npz', model)

In [7]:
X_train = model.encode(xnum_train)[0].data
X_test = model.encode(xnum_test)[0].data

# Prediction

In [15]:
MLP = MLPRegressor((1000, 1000, 1000))
MLP.fit(X_train, y_train)

y_train_pred = MLP.predict(X_train)
print("Train R2: {:.4f}".format(r2_score(y_train, y_train_pred)))
print("Train MSE: {:.4f}".format(mean_squared_error(y_train, y_train_pred)))

y_pred = MLP.predict(X_test)
print("Test R2: {:.4f}".format(r2_score(y_test, y_pred)))
print("Test MSE: {:.4f}".format(mean_squared_error(y_test, y_pred)))

Train R2: 0.9878
Train MSE: 0.0504
Test R2: 0.7176
Test MSE: 1.1529


In [16]:
1.1529**0.5

1.0737318100903968

In [10]:
param_grid = { "hidden_layer_sizes": [(100,100,100), (500,500,500), (1000,1000,1000), (100,100,100,100), (500, 500,500,500), (1000,1000,1000,1000)]}
MLP_grid = GridSearchCV(estimator=MLPRegressor(), param_grid=param_grid, cv=5, n_jobs=8, verbose=1)
MLP_grid.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  30 out of  30 | elapsed:  2.9min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False),
       fit_params=None, iid='warn', n_jobs=8,
       param_grid={'hidden_layer_sizes': [(100, 100, 100), (500, 500, 500), (1000, 1000, 1000), (100, 100, 100, 100), (500, 500, 500, 500), (1000, 1000, 1000, 1000)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [11]:
MLP_grid.best_params_

{'hidden_layer_sizes': (1000, 1000, 1000, 1000)}

In [12]:
MLP_grid.best_estimator_.fit(X_train, y_train)
y_train_pred = MLP_grid.best_estimator_.predict(X_train)
y_test_pred = MLP_grid.best_estimator_.predict(X_test)

print('MSE train : %.3f, test : %.3f' % (mean_squared_error(y_train, y_train_pred), mean_squared_error(y_test, y_test_pred)) )
print('R2 train : %.3f, test : %.3f' % (r2_score(y_train, y_train_pred), r2_score(y_test, y_test_pred)) )

MSE train : 0.053, test : 1.167
R2 train : 0.987, test : 0.714


In [12]:
RF = RandomForestRegressor()
RF.fit(X_train, y_train)

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [13]:
y_train_pred = RF.predict(X_train)
y_test_pred = RF.predict(X_test)

print('MSE train : %.3f, test : %.3f' % (mean_squared_error(y_train, y_train_pred), mean_squared_error(y_test, y_test_pred)) )
print('R2 train : %.3f, test : %.3f' % (r2_score(y_train, y_train_pred), r2_score(y_test, y_test_pred)) )

MSE train : 0.285, test : 1.418
R2 train : 0.931, test : 0.653


In [14]:
param_grid = { "max_depth": [2,5,10, None],
                "n_estimators": [10,50,100,300],
                "max_features": [1, 3, 10],
                "min_samples_split": [2, 3, 10],
                "min_samples_leaf": [1, 3, 10]
}

In [15]:
RF_grid = GridSearchCV(estimator=RandomForestRegressor(), param_grid=param_grid, cv=5, n_jobs=8, verbose=1)
RF_grid.fit(X_train, y_train)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    2.4s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    6.9s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:   14.6s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:   26.4s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:   45.4s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:  1.2min
[Parallel(n_jobs=8)]: Done 2160 out of 2160 | elapsed:  1.5min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=8,
       param_grid={'max_depth': [2, 5, 10, None], 'n_estimators': [10, 50, 100, 300], 'max_features': [1, 3, 10], 'min_samples_split': [2, 3, 10], 'min_samples_leaf': [1, 3, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [16]:
RF_grid.best_params_

{'max_depth': None,
 'max_features': 10,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 300}

In [17]:
RF_grid.best_estimator_.fit(X_train, y_train)
y_train_pred = RF_grid.best_estimator_.predict(X_train)
y_test_pred = RF_grid.best_estimator_.predict(X_test)

print('MSE train : %.3f, test : %.3f' % (mean_squared_error(y_train, y_train_pred), mean_squared_error(y_test, y_test_pred)) )
print('R2 train : %.3f, test : %.3f' % (r2_score(y_train, y_train_pred), r2_score(y_test, y_test_pred)) )

MSE train : 0.204, test : 1.339
R2 train : 0.951, test : 0.672
